In [103]:
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from datasets import load_dataset
import pandas as pd
import ast
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score




In [104]:
#dowlnoad the dataset
data_load = load_dataset("nedjmaou/MLMA_hate_speech")
data_train = data_load["train"]
dataset = data_train.to_pandas()
print(dataset.head())
print(dataset.shape)

   HITId                                              tweet       sentiment  \
0      1  صلاة الفجر خير لك من ترديد بول البعير وسبي الن...  hateful_normal   
1      2  صراحة نفسي اشوف ولاد الوسخة اللي قالوا مدرب اج...       offensive   
2      3  طيب! هي متبرجة وعبايتها ملونه وطالعة من بيتهم ...       offensive   
3      4  @user @user انا اوافقك بخصوص السوريين و العراق...          normal   
4      5  هذه السعودية التي شعبها شعب الخيم و بول البعير...          normal   

  directness                           annotator_sentiment  target       group  
0   indirect                                         shock  gender  individual  
1   indirect  anger_confusion_sadness_indifference_disgust   other       other  
2   indirect                                  indifference   other  individual  
3     direct                                  indifference  origin       other  
4   indirect                                  indifference  origin       other  
(18661, 7)


In [172]:
#dataset separate by languages
arab_path = "/Users/benoitmathey-doret/Documents/EPFL/MA2/DL/project/deeplearning/dataset/ar_dataset_600.csv"
ar_dataset = pd.read_csv(arab_path)
print(ar_dataset.shape)
print(ar_dataset.head())
ar_dataset = ar_dataset.sample(frac=1, random_state=42)



en_path="/Users/benoitmathey-doret/Documents/EPFL/MA2/DL/project/deeplearning/dataset/en_dataset_600.csv"
en_dataset = pd.read_csv(en_path)
print(en_dataset.shape)
print(en_dataset.head())
en_dataset = en_dataset.sample(frac=1, random_state=42)

en_dataset_binary=en_dataset.copy()
en_dataset_binary["sentiment_binary"]=en_dataset_binary["sentiment_binary"].replace(3,2)

fr_path="/Users/benoitmathey-doret/Documents/EPFL/MA2/DL/project/deeplearning/dataset/fr_dataset_600.csv"
fr_dataset = pd.read_csv(fr_path)
print(fr_dataset.shape)
print(fr_dataset.head())
fr_dataset = fr_dataset.sample(frac=1, random_state=42)



(600, 8)
   HITId                                              tweet sentiment  \
0   1486                          بول البعير وعلاج السرطان!    normal   
1   3194  الإسلام يتعامل مع القضايا قبل حصولها فيسعى لعل...    normal   
2   1384  @user @user معلش سؤال..الكائن الابيض دا نوعه ا...    normal   
3   1391  محاضرة التحرش والابتزاز كلية الحقوق والعلوم ال...    normal   
4   1420  حرب شرسه على المرآه وحجابها من كل مكان الاعلام...    normal   

  directness   annotator_sentiment    target  group  sentiment_binary  
0   indirect  indifference_disgust     other  other                 1  
1   indirect          indifference  religion  other                 1  
2   indirect                 shock    origin  other                 1  
3   indirect          indifference     other  other                 1  
4     direct               sadness     other  women                 1  
(600, 8)
   HITId                                              tweet sentiment  \
0   5421                              

In [158]:
#One by One

used_data = en_dataset_binary.iloc[0:40]
predictions_concat=[]

# Wrap range with tqdm
for i in tqdm(range(used_data.shape[0])):
    tweet = en_dataset['tweet'].iloc[i]
    # prompt
    prompt = [{"role": "system", "content": "Do you think these tweets are hate speech or normal speech. Try your best even if we do not have context. Display the reasult as an integer of 1 (Normal) or 2 (Hateful). Do not display anything else, it is important that only the interger is displayed."},
              #{"role": "system", "content": "Do you think these tweets are hate speech, offensive or normal speech. Try your best even if we do not have context. Display the reasult as an integer of 1 (Normal), 2(offensive) or 3 (Hateful). Do not display anything else, it is important that only the interger is displayed."},
              {"role": "user", "content": tweet},]
    # LLM initialization
    llm = Ollama(model="llama3")
    predictions = llm.invoke(prompt)
    predictions = int(predictions) 
    predictions_concat.append(predictions)

print(len(predictions_concat))
#add the predictions to the dataset
used_data['predictions'] = predictions_concat


  0%|          | 0/40 [00:00<?, ?it/s]

100%|██████████| 40/40 [00:28<00:00,  1.42it/s]

40



/var/folders/gm/941r48zd5qxcl4tb3v9q30440000gn/T/ipykernel_93745/3750342181.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  used_data['predictions'] = predictions_concat


In [159]:
#Metrics
acc=accuracy_score(used_data['sentiment_binary'], used_data['predictions'])
F1=f1_score(used_data['sentiment_binary'], used_data['predictions'], average='macro')
Precision=precision_score(used_data['sentiment_binary'], used_data['predictions'], average='macro')
Recall=recall_score(used_data['sentiment_binary'], used_data['predictions'], average='macro')
print("Accuracy: ", acc, "F1: ", F1, "Precision: ", Precision, "Recall: ", Recall)

Accuracy:  0.775 F1:  0.7565922920892495 Precision:  0.8166144200626959 Recall:  0.7550505050505051


In [160]:
print(used_data[[ 'sentiment','sentiment_binary', 'predictions']])

     sentiment  sentiment_binary  predictions
110     normal                 1            1
419    hateful                 2            2
565    hateful                 2            2
77      normal                 1            1
181     normal                 1            2
284  offensive                 2            2
10      normal                 1            2
469    hateful                 2            2
78      normal                 1            1
349  offensive                 2            2
55      normal                 1            2
118     normal                 1            2
109     normal                 1            1
588    hateful                 2            2
369  offensive                 2            2
234  offensive                 2            2
30      normal                 1            1
212  offensive                 2            1
184     normal                 1            2
86      normal                 1            2
2       normal                 1  

In [173]:
print(en_dataset['sentiment_binary'].unique())

[1 3 2]
